#### ***Importing Necessary Libraries*** :

In [12]:
#For Model interpretation
!pip install lime
!pip install shap

#install xgboost
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 275 kB 6.7 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=52f39fef4c8d33d06f8a31827ff96487426aa742f79ea419c93d1cb4c4e8aa1b
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 7.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import numpy as np # mathematical computation
import pandas as pd # data processing
import matplotlib.pyplot as plt # visualization
from matplotlib import pyplot
import seaborn as sns # visualization
import warnings 
warnings.filterwarnings('ignore') #ignore warnings
from io import BytesIO
import datetime
import os

#set style
plt.style.use('seaborn')

# Setting font weight,label weight,title weight to bold and setting title size,label size,fontsize.
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["axes.titlesize"] = 15
plt.rcParams["axes.titleweight"] = 'bold'
plt.rcParams['xtick.labelsize']=10
plt.rcParams['ytick.labelsize']=10
plt.rcParams["axes.labelsize"] = 15
plt.rcParams["legend.fontsize"] = 10
plt.rcParams["legend.title_fontsize"] = 15

#imports for data preparationa and modeling
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score,mean_absolute_error
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

#import for creating tree
from sklearn import tree
from sklearn.tree import plot_tree, export_text
#import pickle Package
import pickle

#imports for model interpretation
import lime
from lime import lime_tabular
import shap

#### ***Importing Bike share rides data*** :

In [ ]:

from google.colab import files
uploaded = files.upload()

#### ***Reading CSV file into dataframe*** :


In [ ]:
Bikes_df=pd.read_csv(BytesIO(uploaded['SeoulBikeData.csv']),encoding = 'unicode_escape')

#### ***Data Exploration And Data Cleaning*** :

In [ ]:
# Check sample 5 rows
Bikes_df.sample(5)

In [ ]:
# overview of dimensions  and datatypes of the dataset 
print(f'The shape of dataset is {(Bikes_df.shape)} \n Total Rows are : {(Bikes_df.shape)[0]}\n Total Columns are : {(Bikes_df.shape)[1]}')
print('\n')
Bikes_df.info()

In [ ]:
#Dealing with invalid data types
Bikes_df['Date']=Bikes_df['Date'].astype('datetime64[ns]')
Bikes_df['Seasons']=Bikes_df['Seasons'].astype('category')
Bikes_df['Holiday']=Bikes_df['Holiday'].astype('category')
Bikes_df['Hour']=Bikes_df['Hour'].astype('category')
Bikes_df['Functioning Day']=Bikes_df['Functioning Day'].astype('category')
# Renaming columns for better analysis
Bikes_df.rename(columns={'Rented Bike Count':'Rented_Bike_Count','Temperature(°C)':'Temperature','Humidity(%)':'Humidity','Wind speed (m/s)':'Wind_speed',
                       'Visibility (10m)':'Visibility','Dew point temperature(°C)':'Dew_point_temperature', 'Solar Radiation (MJ/m2)':'Solar_Radiation',
                        'Rainfall(mm)':'Rainfall','Snowfall (cm)':'Snowfall','Functioning Day':'Functioning_Day'},inplace=True)

#Extracting Month and Day of the week from Date
Bikes_df['Month']=Bikes_df['Date'].dt.month_name().astype('category')
Bikes_df['Day']=Bikes_df['Date'].dt.day_name().astype('category')
Bikes_df["year"] = Bikes_df['Date'].map(lambda x: x.year).astype("category")

#Adding Weekend column from day as 1 for weekedn and 0 for weekday
Bikes_df['Weekend']=Bikes_df['Day'].apply(lambda x: 1 if x=='Saturday' or x=='Sunday' else 0)
Bikes_df['Weekend']=Bikes_df['Weekend'].astype('category')
# drop the Date column
Bikes_df.drop(columns=['Date'],inplace=True)



In [ ]:
# Checking for missing values
print(f'Null values :\n{Bikes_df.isnull().sum()}')
print('\n')
#Checking for duplicate values
print(f'Duplicate values : {sum(Bikes_df.duplicated())}')

In [ ]:
Bikes_df.describe().style.background_gradient()

#### ***Exploratory Data Analysis :***

In [ ]:
Bikes_df.info()

***When are the most rides happening ?*** 📅

In [ ]:
numeric_features= Bikes_df.select_dtypes(exclude=['category','datetime64'])
categorical_features=Bikes_df.select_dtypes(include='category')
# Plot barchart plot for each categorical columns with respect to target variable to get insights
for col in categorical_features:
    fig = plt.figure(figsize=(12, 9))
    ax = fig.gca()
    sns.barplot(x=Bikes_df[col],y=Bikes_df['Rented_Bike_Count'])



>***Observations:***

*  ***Time of day:*** It is evident that the peaks are observed in mornings at 8AM & 9AM and evenings at 4PM, 5PM & 6PM implicating that majority of the trips are taken before and after the usual office hours. Moreover, the trips taken in between the usual office hours i.e. 10AM — 3PM are constant. Additionaly, the trips have declined substantially after 6PM.

*   ***Day of week:*** As majority of the trips were taken before and after the usual office hours, it is no surprise that majority of the trips have taken place at weekdays (Mon-Fri) as compared to weekends (Sat-Sun)

*   ***Season :*** We observe that summer has the highest rentals followed by spring and then fall, which gives us reason to believe that the bike riders prefer warm to pleasant climates than colder climates.









##### ***What is the distribution of rides on different kinds of days wrt Hours?***

In [ ]:
for i in categorical_features:
  if i == 'Hour':
    pass
  else:
    plt.figure(figsize=(20,10))
    sns.pointplot(x=Bikes_df["Hour"],y=Bikes_df['Rented_Bike_Count'],hue=Bikes_df[i])
    plt.title(f"Rented Bike Count during different {i} with respect of Hour")
  plt.show()



> ***Observations :***


* We can observe that the pattern of weekdays and weekends as well as Holidays is different, in the weekend and on holiday the demand becomes high in the afternoon. While the demand for office timings is high during weekdays and no holidays.

* In the month column, We can clearly see that the demand is low in December January & Febuary,  as it is cold in these months and we have already seen in season column that demand is less in winters.

*  Comparitively the demand was high in 2018 than in 2017, which can support the fact that bike share demand is increasing over the years.






#### ***What is the distribution and correlation among variables ?***

In [ ]:
#plotting histogram with mean and median, and distplot of all the numeric features of the dataset
n=1
for i in numeric_features.columns:
  plt.figure(figsize=(20,40))
  plt.subplot(9,2,n)
  n+=1
  print('\n')
  print('*'*70,i,'*'*70)
  print('\n')
  # fig=plt.figure()
  # ax=fig.gca()
  feature=Bikes_df[i]
  feature.hist(bins=50,)
  plt.axvline(feature.mean(), color='purple', linestyle='dashed', linewidth=2)
  plt.axvline(feature.median(), color='coral', linestyle='dashed', linewidth=2) 
  plt.subplot(9,2,n)
  n+= 1
  sns.distplot(Bikes_df[i])
  # plt.tight_layout()
  plt.show()

In [ ]:
# Regression plot to know relation of target variable with our independent variable
n=1
plt.figure(figsize=(15,15))
for i in numeric_features.columns:
  if i == 'Rented_Bike_Count':
    pass
  else:
    plt.subplot(4,2,n)
    n+=1
    sns.regplot(Bikes_df[i], Bikes_df['Rented_Bike_Count'],scatter_kws={"color": "pink"}, line_kws={"color": "green"})
    plt.title(f'Target variable and {i}')
    plt.tight_layout()

In [ ]:
#correlation of all the numerical features with the heat map, so that we will also get to know the multicolinearity
plt.figure(figsize=(8,8))
sns.heatmap(Bikes_df.select_dtypes(include=['float','int']).corr(),annot=True,center = 0)
plt.show()

In [ ]:
#Checking for Data imbalance by creating pieplot
n=1
plt.figure(figsize=(20,15))
columns=['Hour','Seasons','Month']
for i in columns:
  plt.subplot(3,3,n)
  n=n+1
  plt.pie(Bikes_df[i].value_counts(),labels = Bikes_df[i].value_counts().keys().tolist(),autopct='%.0f%%')
  plt.title(i)
  plt.tight_layout()



>***Observations :***


*   From the distplots we could see that some of the variables are either right or lwft skewed and for those variables, even the mean and median are skewed as seen in the histograms.

*  Regression plots show that the few variables are positively correlated and few are nagtively correlated with our target variable.

* From Heat map, we could see that there is multicollineriaty between few variables like Temperature and Dew point temperature.

* From the pie plots , we could see that the data is uniformly distributed across all seasons, months and hours. Hence, there is no data imbalance.





#### ***Preparing Data for Modeling*** :

In [ ]:
# Make copy of dataset
df=Bikes_df.copy()
df

***Multicollinearity*** can be a problem in a regression model because we would not be able to distinguish between the individual effects of the independent variables on the dependent variable.It can be detected via various methods. 
In this project, we used the most common approach ***VIF (Variable Inflation Factors).***

VIF determines the strength of the correlation between the independent variables. It is predicted by taking a variable and regressing it against every other variable.If VIF >5, then variables are highly collinear and the parameter estimates will have large stanhdard errors due to this.








In [ ]:
#Detecting Multicollinearity using VIF(Variable Inflation Factors)
def Calculate_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)
#excluding target variable
Calculate_vif(df[[i for i in df.describe().columns if i not in ['Rented_Bike_Count']]])

In [ ]:
# As per above vif calculation dropping  dew_point_temperature columns.
df.drop(['Dew_point_temperature','Day'],inplace=True,axis=1)
Calculate_vif(df[[i for i in df.describe().columns if i not in ['Rented_Bike_Count']]])

In [ ]:
# Creating dummy variables
df=pd.get_dummies(df,drop_first=True,sparse=True)
# checking modified data
df.head(5)

In [ ]:
# Plot the distribution of the target variable to check for skewness
fig = plt.figure()
sns.distplot((df['Rented_Bike_Count']), bins = 10)


As the target variable is right skewed, we can normalize it by applying  either squareroot or log transformation.

In [ ]:
# log tranformation (adding 1 to bike count to avoid log(0) error)
df['log(Rented_Bike_Count)'] = np.log10(df['Rented_Bike_Count']+1)
fig = plt.figure()
sns.distplot(df['log(Rented_Bike_Count)'], bins = 10)

#square_root transformation
df['sqrt(Rented_Bike_Count)'] = np.sqrt(df['Rented_Bike_Count'])
fig = plt.figure()
sns.distplot(df['sqrt(Rented_Bike_Count)'])

In [ ]:
print(f"Skewness of Target Variable : {df['Rented_Bike_Count'].skew():.2f}")
#print(f"Skewness of Logarithm of Target Variable : {df['log(Rented_Bike_Count)'].skew():.2f}")
print(f"Skewness of Square root of Target Variable : {df['sqrt(Rented_Bike_Count)'].skew():.2f}")

Here, we could see that the target variable is balanced better, after applying square root transformation and the skewness is greatly reduced.

In [ ]:
df.drop(['log(Rented_Bike_Count)','sqrt(Rented_Bike_Count)'],axis=1,inplace=True)

#### ***Defining Function to train the models :***

In [ ]:
# creating a list so that all models evalution scores will be appended to the corrosponding list after hyperarameter
mean_absolute_error_tuned = []
mean_sq_error_tuned=[]
root_mean_sq_error_tuned=[]
Train_accuracy_score_tuned =[]
Test_accuracy_score_tuned =[]
r2_list_tuned=[]
adjusted_r2_list_tuned=[]

# creating a function for fit, predict and evaluting the models and append all evalution score in list 
def model_train(model,X_train,X_test,Y_train,Y_test, linear = False):
  '''
    train the model and gives mae, mse,rmse,r2 and adjusted r2 scores of the model 
  '''
  #training the model
  model.fit(X_train,Y_train)
  # Accuracy Score
  train_accuracy  = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10).mean()
  test_accuracy  = cross_val_score(estimator = model, X = X_test, y = y_test, cv = 10).mean()
  print('*'*30, 'Cross validation Accuracy score', '*'*30)
  print("Train Accuracy score  :", train_accuracy)
  print("Test Accuracy score  :", test_accuracy)

  print('\n')

  try:
      # finding the best parameters of the model if any 
    print('*'*20, 'Best Parameters & Best Score', '*'*20)
    print(f"The best parameter that was found out is :{model.best_params_} \nwhere model's best score is:  {model.best_score_} \n")
  except:
    print('None')
    
    

  #predicting the Test set and evaluting the models 
  print('\n')
  print('*'*20, 'Evalution Metrics', '*'*20)

  if linear == True:
    Y_pred = model.predict(X_test)
    #squaring the target variable test and prediction values as we took y as square root of target variable in modeling.
    #finding mean_absolute_error
    MAE  = mean_absolute_error(Y_test**2,Y_pred**2)
    print("MAE :" , MAE)

    #finding mean_squared_error
    MSE  = mean_squared_error(Y_test**2,Y_pred**2)
    print("MSE :" , MSE)

    #finding root mean squared error
    RMSE = np.sqrt(MSE)
    print("RMSE :" ,RMSE)

    #finding the r2 score
    '''
    R2 assumes that every single variable explains the variation in the dependent variable
    '''
    r2 = r2_score(Y_test**2,Y_pred**2)
    print("R2 :" ,r2)

    #finding the adjusted r2 score
    '''
    The adjusted R2 gives the percentage of variation explained by 
    only the independent variables that actually affect the dependent variable.
    '''
    adj_r2=1-(1-r2_score(Y_test**2,Y_pred**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
    print("Adjusted R2 : ",adj_r2,'\n')
  
  else:
    # for tree base models
    Y_pred = model.predict(X_test)

    #finding mean_absolute_error
    MAE  = mean_absolute_error(Y_test,Y_pred)
    print("MAE :" , MAE)

    #finding mean_squared_error
    MSE  = mean_squared_error(Y_test,Y_pred)
    print("MSE :" , MSE)

    #finding root mean squared error
    RMSE = np.sqrt(MSE)
    print("RMSE :" ,RMSE)

    #finding the r2 score
    
    r2 = r2_score(Y_test,Y_pred)
    print("R2 :" ,r2)
    #finding the adjusted r2 score
    adj_r2=1-(1-r2_score(Y_test,Y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
    print("Adjusted R2 : ",adj_r2,'\n')

    #Top 10 feature importance graph
    try:
      best = model.best_estimator_
      features = new_X.columns
      importances = best.feature_importances_[0:10]
      indices = np.argsort(importances)
      plt.figure(figsize=(10,15))
      plt.title('Feature Importance')
      plt.barh(range(len(indices)), importances[indices], color='blue',edgecolor='pink' ,align='center')
      plt.yticks(range(len(indices)), [features[i] for i in indices])
      plt.xlabel('Relative Importance')
      plt.show()
    
    except:
      pass
  
  # Here we appending the parameters for all models 
  mean_absolute_error_tuned.append(MAE)
  mean_sq_error_tuned.append(MSE)
  root_mean_sq_error_tuned.append(RMSE)
  Train_accuracy_score_tuned.append(train_accuracy)
  Test_accuracy_score_tuned.append(test_accuracy)
  r2_list_tuned.append(r2)
  adjusted_r2_list_tuned.append(adj_r2)

  # print the coefficient and intercept for linear models
  if model == LR:
    print("*"*25, "coefficient", "*"*25)
    print(pd.DataFrame(model.coef_,X.columns, columns=['Coefficient']))
    print('\n')
    print("*"*25, "Intercept", "*"*25)
    print('\n')
    print(model.intercept_)
  else:
    pass
  print('\n')
  
  print('*'*20, 'ploting the graph of actual and predicted only with 100 observation', '*'*20)

  # ploting the graph of Actual and predicted only with 100 observation for better visualisation which model have these parameters and else we just pass them
  try:
    # ploting the line graph of actual and predicted values  
    plt.figure(figsize=(15,7))
    plt.plot((Y_pred)[:100])
    plt.plot((np.array(Y_test)[:100]))
    plt.legend(["Predicted","Actual"])
    plt.show()
  except:
    pass

#### ***Linear models without hyperparameter tuning:***

In [ ]:
mean_abs_error = []
mean_sq_error=[]
root_mean_sq_error=[]
train_accuracy_score =[]
test_accuracy_score =[]
r2_list=[]
adj_r2_list=[]

# Define X-variable(Independent Variables) and Y-Variable(Dependent Variable)
X=df.drop('Rented_Bike_Count',axis=1)
y=np.sqrt(df['Rented_Bike_Count'])
# Split data into train test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
#scaling the features
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
# Checking shape
print(f'Shape of X_train : {X_train.shape}')
print(f'Shape of y_train : {y_train.shape}')
print(f'Shape of X_test : {X_test.shape}')
print(f'Shape of y_test : {y_test.shape}')

#instantiating the model
lr = LinearRegression()
l1 = Lasso()
l2 = Ridge()
e  = ElasticNet()

linear_models = [lr,l1,l2,e]
for i in linear_models:
  print('\n')
  print('*'*20,i,'*'*20) 
  i.fit(X_train,y_train)
  y_pred = i.predict(X_test)
  mae = mean_absolute_error(y_test**2,y_pred**2)
  print("MAE :" , mae)
  mse = mean_squared_error(y_test**2,y_pred**2)
  print("MSE :" , mse)
  rmse = np.sqrt(mse)
  print("RMSE :" , rmse)
  r2 = r2_score(y_test**2,y_pred**2)
  print("R2 :" , r2)
  adj_r2 =1-(1-r2)*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
  print("Adjusted_R2 :" , adj_r2)
  train_accuracy=i.score(X_train,y_train)
  print("Cross validation accuracy:" , train_accuracy)
  test_accuracy=i.score(X_train,y_train)
  print("Cross validation accuracy:" , test_accuracy)


  train_accuracy_score.append(train_accuracy)
  test_accuracy_score.append(test_accuracy)
  mean_abs_error.append(mae)
  mean_sq_error.append(mse)
  root_mean_sq_error.append(rmse)
  r2_list.append(r2)
  adj_r2_list.append(adj_r2)


#### ***Linear Models with Hyperparameter tuning :***

######**Linear Regression**

In [ ]:
LR= LinearRegression()
model_train(LR,X_train,X_test,y_train,y_test,linear=True)

######**Lasso Regression**

In [ ]:
# Using Grid Search CV for Hyperparameter tuning of Lasso Regression model
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100,0.0014]} #lasso parameters 
L1 = GridSearchCV(Lasso(), parameters, cv=5) #using gridsearchcv and cross validation on the model
# fit and evaluate model 
model_train(L1,X_train,X_test,y_train,y_test,linear=True)

######**Ridge Regression**

In [ ]:
# Using Grid Search CV for Hyperparameter tuning for Lasso Regression
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,10,20,30,40,45,50,55,60,100,0.5,1.5,1.6,1.7,1.8,1.9]} # giving parameters 
L2 = GridSearchCV(Ridge(), parameters, cv=5) #using gridsearchcv and cross validate the model
# fit and evaluate model with score_matrix function
model_train(L2,X_train,X_test,y_train,y_test,linear=True)

######**Elastic Regression**

In [ ]:
# Using Grid Search CV for Hyperparameter tuning for Lasso Regression
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,10,20,30,40,45,50,55,60,100,0.5,1.5,1.6,1.7,1.8,1.9]} # giving parameters 
E = GridSearchCV(ElasticNet(), parameters, cv=5) #using gridsearchcv and cross validate the model
# fit and evaluate model with score_matrix function
model_train(E,X_train,X_test,y_train,y_test,linear=True)

######**Polynomial Regression**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
P = PolynomialFeatures(degree=2,include_bias = False) #creating variables with degree 4
P_X_train = P.fit_transform(X_train) # fit the train set
P_X_test = P.transform(X_test) #transform the test set
# fit and evaluate model with score_matrix function
model_train(LR,X_train,X_test,y_train,y_test,linear=True)

#### ***Data Splitting for Tree based Models:***
As Desicion trees make no assumptions on relationships between features, so we can use all the independent features because multi collinearity won't affect the model.Also, we are not transforming the target variable here, as it's distribution won't have impact on model accuracy.

In [ ]:
# Creating DataFrame for tree base models
t_df = pd.get_dummies(Bikes_df,columns=['Seasons','Holiday','Functioning_Day','Month','Hour','Weekend'],drop_first=True)
t_df.drop(['Day'],inplace=True,axis=1)
# Split data in X and Y
new_X = t_df.drop(columns='Rented_Bike_Count')
new_y = t_df['Rented_Bike_Count']
# Train test split our data
X_train,X_test,y_train,y_test = train_test_split(new_X,new_y, test_size=0.25)
#Feature scaling
scaler = MinMaxScaler()
X_train  = scaler.fit_transform(X_train) #fitting the X_train
X_test   = scaler.transform(X_test) # transform test set

##### ***Tree Based Models without Hyper Parameter tuning :***

In [ ]:
#instantiating the models
dt= DecisionTreeRegressor()
rf = RandomForestRegressor()
xgb = XGBRegressor(objective ='reg:squarederror')

tree_models = [dt,rf,xgb]
for i in tree_models:
  print('\n')
  print('*'*20,i,'*'*20) 
  i.fit(X_train,y_train)
  y_pred = i.predict(X_test)
  mae = mean_absolute_error(y_test**2,y_pred**2)
  print("MAE :" , mae)
  mse = mean_squared_error(y_test**2,y_pred**2)
  print("MSE :" , mse)
  rmse = np.sqrt(mse)
  print("RMSE :" , rmse)
  r2 = r2_score(y_test**2,y_pred**2)
  print("R2 :" , r2)
  adj_r2 =1-(1-r2)*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
  print("Adjusted_R2 :" , adj_r2)
  train_accuracy=i.score(X_train,y_train)
  print("Train_Accuracy :",train_accuracy)
  test_accuracy=i.score(X_test,y_test)
  print("Test_Accuracy :",test_accuracy)
  train_accuracy_score.append(train_accuracy)
  test_accuracy_score.append(test_accuracy)
  mean_abs_error.append(mae)
  mean_sq_error.append(mse)
  root_mean_sq_error.append(rmse)
  r2_list.append(r2)
  adj_r2_list.append(adj_r2)


#### ***Tree based models with Hyper Parameter tuning :***

##### ***Decision Tree Regression :***

In [ ]:
# Parameters for Decission Tree model
param_grid = {'criterion':['mae','rmse'],
              'splitter':['best','random'],
              'max_depth' : [5,10 ,15],
              'max_features':['auto','log2']
}
# Using Grid Search 
DT= GridSearchCV(DecisionTreeRegressor(),param_grid=param_grid,cv=5)
# fit and evaluate model 
model_train(DT,X_train,X_test,y_train,y_test)

In [ ]:
plt.figure(figsize=(30,15))
plot_tree(dt, feature_names=new_X.columns, max_depth=3, filled=True)

##### ***Random Forest Regressor :***

In [ ]:
param_grid={'n_estimators':[50,100, 150,200,],'max_depth':[3,5,10],'max_features':['auto','sqrt','log2']}
# Using Grid SearchCV 
RF= GridSearchCV(RandomForestRegressor(),param_grid=param_grid,cv=5)
# fit and evaluate model 
model_train(RF,X_train,X_test,y_train,y_test)

##### ***Extreme Gradient Booster Regressor*** :

In [ ]:
param_grid ={
          'subsample': [0.9,0.7,0.5,0.3], 
          'n_estimators': [50,100,150],  
          'max_depth': range(2,8,2), 
          'learning_rate': [0.02,0.04,0.06], 
          'eval_mertric': ['rmse','mse']   
          }

XGB= GridSearchCV(XGBRegressor(objective ='reg:squarederror'),param_grid=param_grid,cv=5)
# fit and evaluate model 
model_train(XGB,X_train,X_test,y_train,y_test)


#### ***Tree Based Model Explainability*** :

##### ***LIME :***

> LIME stands for Local Interpretable Model-agnostic Explanations



In [ ]:
def me_lime(model,num_features,row_number):
  test = pd.DataFrame(X_test)
  lime_explainer = lime_tabular.LimeTabularExplainer(
      training_data = X_train,
      feature_names = new_X.columns.values,
      categorical_features=t_df.select_dtypes(include='category'),
      mode = 'regression',
      verbose=True,
      random_state =42 )

  lime_exp = lime_explainer.explain_instance(
      data_row = new_X.iloc[row_number],
      predict_fn = model.predict,num_features=num_features,
  )
  return lime_exp.show_in_notebook(show_table=True)

In [ ]:
 #Calling lime function and taking only top 5 number of feature and 2nd Row
me_lime(DT.best_estimator_,6,2)
me_lime(RF.best_estimator_,6,2)
me_lime(XGB.best_estimator_,6,2)

#### ***Creating Pickle File to save model***

In [ ]:
# Save the Model to file in the current working directory
Pkl_Filename = "Pickle_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(XGB, file)

#### ***Reusing Model from Pickel File :***

In [ ]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_XGB_Model = pickle.load(file)
Pickled_XGB_Model

# Use the Reloaded Model to Calculate the accuracy score and predict target values
# Calculate the Score 
score = Pickled_XGB_Model.score(X_test, y_test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_XGB_Model.predict(X_test)  

#### ***Evaluation metrics Comparision :***

#####  ***Without Hyper parameter tuning:***

In [ ]:
# creating a Data frame of model scores without hyperparameter tuning
model_names = ['Linear Regression','Ridge','Lasso','Elastic','Decision_Tree','Random_Forest','xg_boost']
#creating dictionary to store all the metrices 
all_metrics={'Mean_Absolute_error':mean_abs_error,'Mean_square_error':mean_sq_error,'Root_Mean_square_error':root_mean_sq_error,'Train_Accuracy_score':train_accuracy_score,'Test_Accuracy_score':test_accuracy_score,'R2':r2_list,'Adjusted_R2':adj_r2_list}
#converting dictionary to dataframe for simple visualization 
metrics=pd.DataFrame.from_dict(all_metrics,orient='index',columns=model_names)
# Adjusted R2 score is set in descending order 
metrics = metrics.transpose().sort_values("Adjusted_R2",ascending=False).reset_index().rename(columns={'index':'model'})
metrics

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(y='Adjusted_R2', x = 'model', data=metrics)
plt.title("Adjusted R2 with respect to Models",fontsize=15)
plt.show()

#####  ***With Hyper parameter tuning:***

In [ ]:
# creating a Data frame of model scores without hyperparameter tuning
model_names = ['Linear Regression','Ridge','Lasso','Elastic','Polynomial','Decision_Tree','Random_Forest','xg_boost']
#creating dictionary to store all the metrices 
all_metrics_tuned={'Mean_Absolute_error':mean_absolute_error_tuned,'Mean_square_error':mean_sq_error_tuned,'Root_Mean_square_error':root_mean_sq_error_tuned,'Train_Accuracy_score':Train_accuracy_score_tuned,'Test_Accuracy_score':Test_accuracy_score_tuned,'R2':r2_list_tuned,'Adjusted_R2':adjusted_r2_list_tuned}
#converting dictionary to dataframe for simple visualization 
metrics_tuned=pd.DataFrame.from_dict(all_metrics_tuned,orient='index',columns=model_names)
# Adjusted R2 score is set in descending order 
metrics_tuned = metrics_tuned.transpose().sort_values("Adjusted_R2",ascending=False).reset_index().rename(columns={'index':'model'})
metrics_tuned

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(y='Adjusted_R2', x = 'model', data=metrics_tuned)
plt.title("Adjusted R2 with respect to Tuned Models",fontsize=15)
plt.show()

### **Reinforcement Learnning Approach :**

##### ***Environment*** :
    The environment module is a bike station object that does the following functions:
    1.generates simulated and real bike stock data
    2.delivers feedback to the RL agent in the form of bike inventory, incentives/penalties,episode termination status 
    3.updates the bike stock in response to the action taken by the RL agent
    4.resets the environment for a new training session.

In [ ]:

import numpy as np
import pandas as pd
import json

class bikestation():
    
    def __init__(self, mode, debug):    
        print("Creating A Bike Station Environment!!!")
        self.mode = mode
        self.seed = np.random.random_integers(0, 10)
        self.num_hours = 23
        self.current_hour = 0
        self.bike_count_sim = self.generate_count(mode)
         # to be reset to original environment after  every episode
        self.bike_count = self.bike_count_sim.copy()
        self.old_count = self.bike_count[0]
        self.new_count = 0
        self.done = False
        self.reward = 0
        self.bike_moved = 0
        self.debug = debug
        self.actions = [-15, -10, -5, 0]
        self.n_actions = len(self.actions)
        #features of the observation: hour, old count, new count
        self.n_features = 1
        self.game_over = False
        if self.debug == True:
            print("Generating Bike count: {}".format(self.mode))
            print("Bike count: {}".format(self.bike_count))
        
    def generate_count(self, mode):
        # generate a list of 24 hourly bike count based on mode
        # mode: linear which is addition of 3 bikes continuously , random is addition of 3 bikes with a random integer between -5 to 5.
        #initialising the bike stock to be 30 
        bike_count = [20]
        if mode == "linear":
            for i in range(1, 24):
                bike_count.append(bike_count[i-1]+3)
        if mode == "random":
            for i in range(1, 24):
                bike_count.append(bike_count[i-1] + 3 + np.random.random_integers(-5, 5))
        return bike_count

    def feedback(self, action):
        # share back count, reward ,and termination status to agent
        if self.debug == True:
            print("Current Hour: {}".format(self.current_hour))
            print("Current count: {}".format(self.bike_count[self.current_hour]))
            print("Bikes Moved in Last Hour: {}".format(self.bike_moved))
            print("Collect {} rewards".format(self.reward))
            print("Will move {} bikes".format(action))
            print("---")
        if action != 0:
            #if there is any rebalancing happening 
            self.update_count(action)
            self.reward = -0.5*np.abs(action)
            #if the count is greater than 50 at any hour
        if self.bike_count[self.current_hour] > 50:
            self.reward = -30
            #if count is less than 50 at any moment
        if self.bike_count[self.current_hour] < 0:
            self.reward = -30
        if self.current_hour == 23:
            #If count is between 0 to 50  at last hour
            if (self.bike_count[self.current_hour] <= 50)&(self.bike_count[self.current_hour] > 0):
                self.reward = 20
            else: 
                self.reward = -20
            self.done = True
            #self.new_count = 'terminal'
            self.game_over = True
        # update to next hour
        if self.current_hour != 23:
            self.update_hour()
            self.old_count = self.bike_count[self.current_hour - 1]
            self.new_count = self.bike_count[self.current_hour]
        return self.current_hour, self.old_count, self.new_count, self.reward, self.done, self.game_over
    
    def get_old_count(self):
        return self.old_count
    
    def update_count(self, num_bike):
        # update bike count based on RL Agent action 
        if self.current_hour != 23:
            for hour in range(self.current_hour+1, len(self.bike_count)):
                self.bike_count[hour] += num_bike
            self.bike_moved = num_bike
        else:
            if self.debug == True:
                print("Last Hour!!! Can't Move Bikes Further!!!")
            pass
        return
    
    def update_hour(self):
        # update current_hour 
        self.current_hour += 1
        if self.debug == True:
            print("Forwarding to Current Hour")
        return
    
    def reset(self):
        if self.debug == True:
            print("Resetting The Environment ...")
        self.num_hours = 23
        self.current_hour = 0
        self.bike_count = self.bike_count_sim.copy()
        self.done = False
        self.reward = 0
        self.bike_moved = 0
        self.old_count = self.bike_count[0]
        self.new_count = 0
        #return (self.current_hour, self.old_count, self.new_count)
        
    def current_count(self):
        return self.bike_count[self.current_hour]
    
    def get_sim_count(self):
        return self.bike_count 

##### ***`Agent :`***

>  Agent is analagous to Bike station Operator.The agent's action is the number of bikes she can move per hour. At any given hour, the agent has the option of moving 0, 5, 10, or 15 bikes. The reward and punishment structure are as follows:

*   -30 if the hourly bike stock falls outside the range [0, 50].
*   +20 if bike stock is in the range [0, 50] at 23 hours; otherwise, -20
*   -0.5  times the number of bikes eliminated every hour.










In [ ]:
import numpy as np
import pandas as pd

class agent():

    def __init__(self, epsilon, lr, gamma, current_count, debug):
        print("Creating an Agent !!!")
        #Defining actions which is count of bikes that can be moved at a certain hour
        self.actions = [-15, -10, -5, 0]
        self.reward = 0
        self.epsilon = epsilon
        self.lr = lr
        self.gamma = gamma
        self.debug = debug
        self.current_count = current_count
        # performance metric
        self.q_table = pd.DataFrame(columns = self.actions, dtype = np.float64)
        self.hourly_action_history = []
        self.hourly_count_history = []
       
    def choose_action(self, s):
        '''
        This funciton choose an action based on Q Table. It also does 
        validation to ensure count will not be negative after moving bikes.
        Input: 
            - s: current bike count
        
        Output:
            - action: number of bikes to move
        '''
        self.check_state_exist(s)
        self.current_count = s
        # find valid action based on current count 
        valid_state_action = self.find_valid_action(self.q_table.loc[s, :])
        valid_state_action = self.q_table.loc[s, :]
        if np.random.uniform() < self.epsilon:
            try:
                # find the action with the highest expected reward
                valid_state_action = valid_state_action.reindex(np.random.permutation(valid_state_action.index))
                action = valid_state_action.idxmax()
            except:
                # if action list is null, default to 0
                action = 0
            if self.debug == True:
                print("Decided to Move: {}".format(action))
        else:
            # randomly choose an action
            try:
                action = np.random.choice(valid_state_action.index)
            except:
                action = 0
            if self.debug == True:
                print("Randomly Move: {}".format(action))
        self.hourly_action_history.append(action)
        self.hourly_count_history.append(s)
        return action

    def learn(self, s, a, r, s_, g):
        '''
        This function updates Q tables after each interaction with the
        environment.
        Input: 
            - s: current bike count
            - a: current action (number of bikes to move)
            - r: reward received from current state
            - s_: new bike count based on bike moved and new count
        Output: None
        '''
        if self.debug == True:
            print("Moved Bikes: {}".format(a))
            print("Old Bike count: {}".format(s))
            print("New Bike count: {}".format(s_))
            print("---")
        self.check_state_exist(s_)
        q_predict = self.q_table.loc[s, a]
        if g == False:
            # Updated Q Target Value if it is not end of day  
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()
        else:
            # Update Q Target Value as Immediate reward if end of day
            q_target = r
        self.q_table.loc[s, a] += self.lr * (q_target - q_predict)
        return

    def check_state_exist(self, state):
        # Add a new row with state value as index if not exist
        if state not in self.q_table.index:
            self.q_table = self.q_table.append(
                pd.Series(
                        [0]*len(self.actions), 
                        index = self.q_table.columns,
                        name = state
                        )
                )
        return
    
    def find_valid_action(self, state_action):
        '''
        This function check the validity acitons in a given state.
        Input: 
            - state_action: the current state under consideration
        Output:
            - state_action: a pandas Series with only the valid actions that
                            will not cause negative count
        '''
        # remove action that will count to be negative
        for action in self.actions:
            if self.current_count + action < 0:
                if self.debug == True:
                    print("Drop action {}, current count {}".format(action, self.current_count))
                state_action.drop(index = action, inplace = True)
        return state_action
        
    
    def print_q_table(self):
        print(self.q_table)

    def get_q_table(self):
        return self.q_table

    def get_hourly_actions(self):
        return self.hourly_action_history
    
    def get_hourly_counts(self):
        return self.hourly_count_history

    def reset_hourly_history(self):
        self.hourly_action_history = []
        self.hourly_count_history = []



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

class trainer():
    def __init__(self):
        # Session Properties
        self.episodes = []
        self.count_type = ""
        self.logging = False
        self.env_debug = False
        self.rl_debug = False
        self.bike_station = None
        self.operator = None
        self.sim_count = []
        # Performance Metric
        self.success_ratio = 0
        self.rewards = []  # [[r from session 1], [r from session 2] ...]
        self.avg_rewards = [] #[np.mean([r from session 1]), np.mean([r from session 2])...]
        self.final_counts = [] # [[count from session 1], [count from session 2] ...]
        self.episode_action_history = []
        self.episode_count_history = []
        self.session_action_history = []
        self.session_count_history = []
        self.q_tables = []
        self.actions = [-15, -10, -5, 0]
        
    def start(self, episodes, count_type, logging, env_debug, rl_debug):
        self.episodes = episodes
        self.count_type = count_type
        self.logging = logging
        self.env_debug = env_debug
        self.rl_debug = rl_debug
        idx = 0
        for eps in self.episodes:
            # Initiate new evironment and RL agent
            self.bike_station = bikestation(self.count_type, debug = self.env_debug)
            self.sim_count.append(self.bike_station.get_sim_count())

            self.operator = agent(epsilon = 0.9, lr = 0.01, gamma = 0.9, current_count = self.bike_station.current_count(), debug = self.rl_debug)
            # Train the RL agent and collect performance stats
            rewards, final_counts = self.train_operator(idx, len(self.episodes), eps, logging = self.logging)
            # Log the results from this training session
            self.rewards.append(rewards)
            self.avg_rewards.append(np.mean(rewards))
            self.final_counts.append(final_counts)
            #self.q_tables.append(self.operator.get_q_table())
            self.session_action_history.append(self.episode_action_history)
            self.session_count_history.append(self.episode_count_history)
            self.reset_episode_history()
            # Destroy the environment and agent objects
            self.bike_station = None
            self.operator = None
            idx += 1
        if logging == True:
            self.save_session_results(self.get_timestamp(replace = True))
        return
    
    def train_operator(self, idx, num_sessions, episodes, logging):
        '''
        This function trains an RL agent by interacting with the bike station 
        environment. It also tracks and reports performance stats.
        Input:
            - episodes: a int of episode to be trained in this session (e.g. 500)
        Output:
            - reward_list: a list of reward per episode in this sesison
            - final_counts: a list of final counts per episode in this session
        '''
        print("Start training the Agent !!!")
        rewards = 0
        reward_list = []
        final_counts = []
        step = 0
        for eps in range(episodes):
            self.bike_station.reset()
            while True:
                # Agent picks an action (number of bikes to move)
                # Agent sends the action to bike station environment
                # Agent gets feedback from the environment (e.g. reward of the action, new bike count after the action, etc.)
                # Agent "learn" the feedback by updating its Q-Table (state, action, reward)
                # Repeat until end of day (23 hours)
                # Reset bike station environment to start a new day, repeat all
                action = self.operator.choose_action(self.bike_station.get_old_count())
                current_hour, old_count, new_count, reward, done, game_over = self.bike_station.feedback(action)
                #observation_, reward, done = self.bike_station.feedback(action)
                if done == True:
                    print("{} of {} Session | Episode: {} | Final count: {} |Final Reward: {:.2f}".format(idx, 
                          num_sessions, eps, old_count, rewards))
                    reward_list.append(rewards)
                    final_counts.append(old_count)
                    rewards = 0
                    # Log session action history by episode
                    self.episode_action_history.append(self.operator.get_hourly_actions())
                    self.episode_count_history.append(self.operator.get_hourly_counts())
                    self.operator.reset_hourly_history()               
                    break
                self.operator.learn(old_count, action, reward, new_count, game_over)
                step +=1
                rewards += reward                    
        return reward_list, final_counts
    
    def get_timestamp(self, replace):
        if replace == True:
            return str(datetime.datetime.now()).replace(" ", "").replace(":", "").\
                        replace(".", "").replace("-", "")
        else:
            return str(datetime.datetime.now())
    
    def reset_episode_history(self):
        self.episode_action_history = []
        self.episode_count_history = []
        
    def cal_performance(self):
        successful_counting = []
        print("===== Performance =====")
        for session in range(len(self.final_counts)):
            length = len(self.final_counts[session])
            num_overcount = np.count_nonzero(np.array(self.final_counts[session]) > 50)
            num_undercount = np.count_nonzero(np.array(self.final_counts[session]) <= 0)
            ratio = (length - num_undercount - num_overcount)*100 / length
            print("Session {} | Overcount {} Times | Undercount {} Times | {}% Successful".format(session, num_overcount, 
                  num_undercount, ratio))
            average_reward = round(self.avg_rewards[session], 2)
            print("Average Episode Reward for Session: {}".format(average_reward))
            successful_counting.append(ratio)
        return successful_counting
    
    def save_session_results(self, timestamp):
        '''
        This function logs the following: 
            - overall success ratio of each session
            - Q Table of each session
        '''
        # --- create a session folder ---
        dir_path = "./performance_log/" + timestamp
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        successful_counting = self.cal_performance()
        # --- Write Success Rate to File ---
        fname = dir_path + "/success_rate - " + timestamp + ".txt"
        with open(fname, 'w') as f:
            f.write("Logged at {}".format(self.get_timestamp(replace = False)))
            f.write("\n")
            f.write("This training session ran episodes: {}".format(self.episodes))
            f.write("\n")
            for session in range(len(successful_counting)):
                f.write("Session {} | Episodes: {} | Success Rate: {:.2f}%".format(session, 
                        self.episodes[session], successful_counting[session]))
                f.write("\n")
        # --- Save Q tables --- 
        for session in range(len(self.q_tables)):
            self.q_tables[session].to_csv(dir_path + "/q_table_session_" + \
                        str(session) + timestamp + ".csv")


episode_list = [eps for eps in range(2000, 8000, 2000)]
data = input("Linear or Random?: ").lower()
if __name__ == "__main__":
    trainer = trainer()
    trainer.start(episode_list, data, logging = True, env_debug = False, rl_debug = False)